In [1]:
from pathlib import Path
import random
from typing import Dict, List, Union

from cloudpathlib import S3Path
import geopandas as gpd
import pandas as pd
import rasterio
import pandas as pd

import os
import re

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC, SDS
import pyproj
from pyproj import CRS, Proj
import shapely.wkt

### Import Training Files

In [2]:
maiac_md = pd.read_csv(
    "./satellite_metadata.csv",
    parse_dates=["time_start", "time_end"],
    index_col=0
)

grid_md = pd.read_csv(
    "./grid_metadata.csv",
    index_col=0
)

### Add Long + Lat to Training Files

In [3]:
grid_bounds = []
for g in  grid_md['wkt']:
    P = shapely.wkt.loads(g)
    coords = P.exterior.coords
    lat_min = min(coords[0][1], coords[1][1], coords[2][1], coords[3][1], coords[4][1])
    lat_max = max(coords[0][1], coords[1][1], coords[2][1], coords[3][1], coords[4][1])
    long_min = min(coords[0][0], coords[1][0], coords[2][0], coords[3][0], coords[4][0])
    long_max = max(coords[0][0], coords[1][0], coords[2][0], coords[3][0], coords[4][0])
    grid_bounds.append([lat_min, lat_max, long_min, long_max])

In [4]:
grid_md['grid_bounds'] = grid_bounds

In [5]:
def transform_arrays(
    xv: Union[np.array, float],
    yv: Union[np.array, float],
    crs_from: CRS,
    crs_to: CRS
):
    """Transform points or arrays from one CRS to another CRS.
    
    Args:
        xv (np.array or float): x (longitude) coordinates or value.
        yv (np.array or float): y (latitude) coordinates or value.
        crs_from (CRS): source coordinate reference system.
        crs_to (CRS): destination coordinate reference system.
    
    Returns:
        lon, lat (tuple): x coordinate(s), y coordinate(s)
    """
    transformer = pyproj.Transformer.from_crs(
        crs_from,
        crs_to,
        always_xy=True,
    )
    lon, lat = transformer.transform(xv, yv)
    return lon, lat

In [6]:
def create_meshgrid(alignment_dict: Dict, shape: List[int]):
    """Given an image shape, create a meshgrid of points
    between bounding coordinates.
    
    Args:
        alignment_dict (Dict): dictionary containing, at a minimum,
            `upper_left` (tuple), `lower_right` (tuple), `crs` (str),
            and `crs_params` (tuple).
        shape (List[int]): dataset shape as a list of
            [orbits, height, width].
    
    Returns:
        xv (np.array): x (longitude) coordinates.
        yv (np.array): y (latitude) coordinates.
    """
    # Determine grid bounds using two coordinates
    x0, y0 = alignment_dict["upper_left"]
    x1, y1 = alignment_dict["lower_right"]
    
    # Interpolate points between corners, inclusive of bounds
    x = np.linspace(x0, x1, shape[2], endpoint=True)
    y = np.linspace(y0, y1, shape[1], endpoint=True)
    
    # Return two 2D arrays representing X & Y coordinates of all points
    xv, yv = np.meshgrid(x, y)
    return xv, yv

In [7]:
# Loop over orbits to apply the attributes
def calibrate_data(dataset: SDS, shape: List[int], calibration_dict: Dict):
    """Given a MAIAC dataset and calibration parameters, return a masked
    array of calibrated data.
    
    Args:
        dataset (SDS): dataset in SDS format (e.g. blue band AOD).
        shape (List[int]): dataset shape as a list of [orbits, height, width].
        calibration_dict (Dict): dictionary containing, at a minimum,
            `valid_range` (list or tuple), `_FillValue` (int or float),
            `add_offset` (float), and `scale_factor` (float).
    
    Returns:
        corrected_AOD (np.ma.MaskedArray): masked array of calibrated data
            with a fill value of nan.
    """
    corrected_AOD = np.ma.empty(shape, dtype=np.double)
    for orbit in range(shape[0]):
        data = dataset[orbit, :, :].astype(np.double)
        invalid_condition = (
            (data < calibration_dict["valid_range"][0]) |
            (data > calibration_dict["valid_range"][1]) |
            (data == calibration_dict["_FillValue"])
        )
        data[invalid_condition] = np.nan
        data = (
            (data - calibration_dict["add_offset"]) *
            calibration_dict["scale_factor"]
        )
        data = np.ma.masked_array(data, np.isnan(data))
        corrected_AOD[orbit, : :] = data
    corrected_AOD.fill_value = np.nan
    return corrected_AOD

In [8]:
def contstruct_metadata_dict(hdf):
    # Construct grid metadata from text blob
    raw_attr = hdf.attributes()["StructMetadata.0"]
    
    group_1 = raw_attr.split("END_GROUP=GRID_1")[0]
    hdf_metadata = dict([x.split("=") for x in group_1.split() if "=" in x])

    # Parse expressions still wrapped in apostrophes
    for key, val in hdf_metadata.items():
        try:
            hdf_metadata[key] = eval(val)
        except:
            pass
    
    return hdf_metadata

In [9]:
def convert_array_to_df(
    corrected_arr: np.ma.MaskedArray,
    lat:np.ndarray,
    lon: np.ndarray,
    granule_id: str,
    crs: CRS,
    total_bounds: np.ndarray = None
):
    """Align data values with latitude and longitude coordinates
    and return a GeoDataFrame.
    
    Args:
        corrected_arr (np.ma.MaskedArray): data values for each pixel.
        lat (np.ndarray): latitude for each pixel.
        lon (np.ndarray): longitude for each pixel.
        granule_id (str): granule name.
        crs (CRS): coordinate reference system
        total_bounds (np.ndarray, optional): If provided,
            will filter out points that fall outside of these bounds.
            Composed of xmin, ymin, xmax, ymax.
    """
    lats = lat.ravel()
    lons = lon.ravel()
    n_orbits = len(corrected_arr)
    size = lats.size
    values = {
        "value": np.concatenate([d.data.ravel() for d in corrected_arr]),
        "lat": np.tile(lats, n_orbits),
        "lon": np.tile(lons, n_orbits),
        "orbit": np.arange(n_orbits).repeat(size),
        "granule_id": [granule_id] * size * n_orbits
        
    }
    
    df = pd.DataFrame(values).dropna()
    if total_bounds is not None:
        x_min, y_min, x_max, y_max = total_bounds
        df = df[df.lon.between(x_min, x_max) & df.lat.between(y_min, y_max)]
    
    gdf = gpd.GeoDataFrame(df)
    gdf["geometry"] = gpd.points_from_xy(gdf.lon, gdf.lat)
    gdf.crs = crs
    return gdf[["granule_id", "orbit", "geometry", "value"]].reset_index(drop=True)

In [10]:
#def create_satellite_feature(filename):
def get_lon_lat(filename, grid_coords): 

    hdf = SD(filename, SDC.READ)
    blue_band_AOD = hdf.select("Optical_Depth_047")
    name, num_dim, shape, types, num_attr = blue_band_AOD.info()
    calibration_dict = blue_band_AOD.attributes()

    hdf_metadata = contstruct_metadata_dict(hdf)

    # Note that coordinates are provided in meters
    alignment_dict = {
        "upper_left": hdf_metadata["UpperLeftPointMtrs"],
        "lower_right": hdf_metadata["LowerRightMtrs"],
        "crs": hdf_metadata["Projection"],
        "crs_params": hdf_metadata["ProjParams"]
    }
    
    corrected_aod = calibrate_data(blue_band_AOD, shape, calibration_dict) # first image


    current_min = np.inf
    best_band = 0
    for idx, band in enumerate(corrected_aod):
        total_missing = np.sum(band.mask)
        if total_missing <= current_min:
            best_band = idx 
            current_min = min(current_min, total_missing)
    
    corrected_aod = corrected_aod[best_band]
        

    xv, yv = create_meshgrid(alignment_dict, shape)

    # Source: https://spatialreference.org/ref/sr-org/modis-sinusoidal/proj4js/
    sinu_crs = Proj(f"+proj=sinu +R={alignment_dict['crs_params'][0]} +nadgrids=@null +wktext").crs
    wgs84_crs = CRS.from_epsg("4326")
    lon, lat = transform_arrays(xv, yv, sinu_crs, wgs84_crs)
    
    
    # Filter to Grid Only
    min_lat = grid_coords[0]
    max_lat = grid_coords[1]
    min_long = grid_coords[2]
    max_long = grid_coords[3]
    
    good_lat_coords = np.argwhere((lat[:, 0]>=min_lat) & (lat[:, 0]<=max_lat))
    good_long_coords = np.argwhere((lon[0, :]>=min_long) & (lon[0, :]<=max_long))

    # Use Bands Plus 3 buffer range
    coords = [min(good_lat_coords)[0]-3, max(good_lat_coords)[0]+3, min(good_long_coords)[0]-3, max(good_long_coords)[0]+3]
    splice = corrected_aod[min(good_lat_coords)[0]-3:max(good_lat_coords)[0]+3, min(good_long_coords)[0]-3:max(good_long_coords)[0]+3]
    
    return np.mean(splice.data), coords
